#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter CM To BigQuery IO Synch Parameters
Migrate CM Placements to DV360 IO.
 1. Fill in the CM accont information.
 1. Fill in the DV360 accont information.
 1. Click the 'Save' button to save configuration.
 1. Click the 'Run Now' button to create the workflow.
 1. Visit <b>BigQuery->UNDEFINED->UNDEFINED->DV360_IO_REVIEW</b> to check for updates.
 1. Visit <b>BigQuery->UNDEFINED->UNDEFINED->DV360_IO_INSERT</b> to approve the updates.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'cm_account': '',  # Comma seperated.
  'dv360_advertiser': '',  # Comma seperated.
  'cm_advertiser': '',  # Comma seperated.
  'dv360_advertisers': '',  # Comma seperated.
  'dv360_campaign': '',  # Comma seperated.
  'recipe_slug': '',  # Place where tables will be created in BigQuery.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute CM To BigQuery IO Synch
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
      'auth': 'user'
    }
  },
  {
    'sheets': {
      'range': 'A1:C',
      'auth': 'user',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
          'table': 'Rules'
        }
      },
      'tab': 'Rules',
      'sheet': 'BB Demo'
    }
  },
  {
    'dv360_api': {
      'auth': 'user',
      'advertisers': {
        'values': {'field': {'name': 'dv360_advertisers','description': 'Comma seperated.','kind': 'integer_list'}},
        'single_cell': True
      },
      'out': {
        'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
        'auth': 'user'
      },
      'endpoints': [
        'advertisers.insertionOrders'
      ]
    }
  },
  {
    'dcm_api': {
      'auth': 'user',
      'advertisers': {
        'values': {'field': {'name': 'cm_advertiser','description': 'Comma seperated.','kind': 'integer_list'}}
      },
      'accounts': {
        'values': {'field': {'name': 'cm_account','description': 'Comma seperated.','kind': 'integer_list'}}
      },
      'out': {
        'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
        'auth': 'user'
      },
      'endpoints': [
        'campaigns',
        'placements',
        'placementGroups'
      ]
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
        'view': 'DV360_IO_LOGIC'
      },
      'from': {
        'legacy': False,
        'query': "SELECT   [PARAMETER] AS advertiserId,   [PARAMETER] AS campaignId,   REGEXP_REPLACE(CM_P.name, '_1X1.*', '') AS displayName,   'ENTITY_STATUS_DRAFT' AS entityStatus,   STRUCT(     'PACING_PERIOD_FLIGHT' AS pacingPeriod,     'PACING_TYPE_AHEAD' AS pacingType   ) AS pacing,   STRUCT(     false AS unlimited,     'TIME_UNIT_DAYS' AS timeUnit,     1 AS timeUnitCount,     CASE       WHEN CM_P.name LIKE '%FF%' AND CM_P.name LIKE '%PZN%'THEN 6       WHEN CM_P.name LIKE '%FF%' THEN 3       WHEN CM_P.name LIKE '%HOL%' THEN 6       WHEN CM_P.name LIKE '%BTC%' THEN 3     END AS maxImpressions   ) AS frequencyCap,   STRUCT(     'PERFORMANCE_GOAL_TYPE_CPM' AS performanceGoalType,     '10' AS performanceGoalAmountMicros   ) AS performanceGoal,   STRUCT(     'BUDGET_UNIT_CURRENCY' AS budgetUnit,     'INSERTION_ORDER_AUTOMATION_TYPE_BUDGET' AS automationType,     [       STRUCT(        '10000' AS budgetAmountMicros,        '' AS description,        STRUCT (          STRUCT (            EXTRACT(YEAR FROM GREATEST(CM_C.startDate, CURRENT_DATE())) AS year,            EXTRACT(MONTH FROM GREATEST(CM_C.startDate, CURRENT_DATE())) AS month,            EXTRACT(DAY FROM GREATEST(CM_C.startDate, CURRENT_DATE())) AS day          ) AS startDate,          STRUCT (            EXTRACT(YEAR FROM CM_C.endDate) AS year,            EXTRACT(MONTH FROM CM_C.endDate) AS month,            EXTRACT(DAY FROM CM_C.endDate) AS day          ) AS endDate        ) AS dateRange       )     ] AS budgetSegments   ) AS budget,   STRUCT(     STRUCT(       '10' AS bidAmountMicros     ) AS fixedBid   ) AS bidStrategy FROM `[PARAMETER].CM_placements` As CM_P LEFT JOIN `[PARAMETER].CM_campaigns` As CM_C ON CM_P.campaignId=CM_C.id LEFT JOIN `[PARAMETER].CM_placementGroups` As CM_G ON CM_P.placementGroupId=CM_G.id WHERE CM_P.advertiserID=4461789 AND CM_P.name LIKE 'PKG%' /*AND CM_G.placementGroupType = 'PACKAGE' */ ORDER BY displayName ;",
        'parameters': [
          {'field': {'name': 'dv360_advertiser','description': 'Comma seperated.','kind': 'integer_list'}},
          {'field': {'name': 'dv360_campaign','description': 'Comma seperated.','kind': 'integer_list'}},
          {'field': {'name': 'recipe_slug','description': 'Place where tables will be written in BigQuery.','kind': 'string'}},
          {'field': {'name': 'recipe_slug','description': 'Place where tables will be written in BigQuery.','kind': 'string'}},
          {'field': {'name': 'recipe_slug','description': 'Place where tables will be written in BigQuery.','kind': 'string'}}
        ]
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
        'view': 'DV360_IO_PREVIEW'
      },
      'from': {
        'legacy': False,
        'query': 'SELECT STRUCT(   ROW_NUMBER() OVER() AS NUMBER,   displayName IN (SELECT displayName FROM `[PARAMETER].DV360_advertisers_insertionOrders`) AS DUPLICATE ) AS PREVIEW, * FROM `[PARAMETER].DV360_IO_LOGIC` ORDER BY displayName ;',
        'parameters': [
          {'field': {'name': 'recipe_slug','description': 'Place where tables will be written in BigQuery.','kind': 'string'}}
        ]
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
        'view': 'DV360_IO_INSERT'
      },
      'from': {
        'legacy': False,
        'query': ' SELECT * EXCEPT(PREVIEW) FROM `[PARAMETER].DV360_IO_PREVIEW` WHERE PREVIEW.NUMBER IN(0,0,0,0,0) AND PREVIEW.DUPLICATE=False LIMIT 1 ;',
        'parameters': [
          {'field': {'name': 'recipe_slug','description': 'Place where tables will be written in BigQuery.','kind': 'string'}}
        ]
      }
    }
  },
  {
    'dv360_api': {
      'insert': 'advertisers.insertionOrders',
      'auth': 'user',
      'bigquery': {
        'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
        'auth': 'user',
        'as_object': True,
        'table': 'DV360_IO_INSERT'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
